<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/80x15.png" /></a><div align="center">This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.</div>

# Data Analysis with Pandas

[Pandas](https://pandas.pydata.org/) is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

We are going to use data produced by the [TissueMAPS](http://tissuemaps.org/) software.

The following data files should be available alongside this notebook:
```
BIO325_CRISPR_Yap_p1_D07_Cells_metadata.csv
BIO325_CRISPR_Yap_p1_D10_Cells_metadata.csv
BIO325_CRISPR_Yap_p1_D07_Cells_feature-values.csv
BIO325_CRISPR_Yap_p1_D10_Cells_feature-values.csv
```

Both *metadata* and *feature-value* [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) files contain data about single cells computed from microscope images; each cell is assigned an ID (unique per "well" -- wells are assigned identifiers like "D07" or "D10"), which matches across the two files.

We can use PanDas to inspect the contents of these files and perform some statistics and plotting.

## Preamble

Before using PanDas, we must import it.  PanDas depends on NumPy and can interchange data with it, so let's import both of them using abbreviated names.

In [1]:
import numpy as np
import pandas as pd

Evaluating the following cell allows us to display plots *in* the Python notebook:

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

# make large figures so we can appreciate details
plt.rc('figure', figsize=(20.0, 15.0))

## Load data

We can use Panda's `read_csv()` function to read a CSV file into a Python data structure: 

In [3]:
# "metadata" file for well D10
md10 = pd.read_csv("BIO325_CRISPR_Yap_p1_D10_Cells_metadata.csv")

In [4]:
# "feature values" for well D10 -- note we read a compressed file!
fv10 = pd.read_csv('BIO325_CRISPR_Yap_p1_D10_Cells_feature-values.csv.gz')

The return object from `read_csv()` is a `DataFrame` object, which displays nicely in the notebook as tabular data:

In [5]:
md10[:5]  # display the first 5 rows

,mapobject_id,plate_name,well_name,well_pos_y,well_pos_x,tpoint,zplane,label,is_border,Classification-5,TPlus
0,491297,p1,D10,0,0,0,0,1,1,0.0,0.0
1,491298,p1,D10,0,0,0,0,2,1,0.0,0.0
2,491299,p1,D10,0,0,0,0,3,1,0.0,0.0
3,491300,p1,D10,0,0,0,0,4,1,0.0,0.0
4,491301,p1,D10,0,0,0,0,5,1,0.0,0.0


Note that the first row of the CSV file was used to name the column, not to provide actual data.

## Access table data

A `DataFrame` object as a `.shape` attribute like any 2D NumPy array:

In [6]:
md10.shape

(30764, 11)

The above shows that our table has 30764 rows across 11 columns.  

**Note: *row index comes first!*** (This will be important when accessing data with numerical indices below.)

### Read entire columns

You can get an *entire column* out of the table, by using Python's `[]` notation with the column name:

In [7]:
labels = md10['label']

In [8]:
labels[:5] # print first 5 label values

0    1
1    2
2    3
3    4
4    5
Name: label, dtype: int64

A column of a PanDas `DataFrame` is a `pandas.Series` object, which closely resembles a NumPy array.

You can use all normal Python and NumPy array methods on it.

In [9]:
len(labels)

30764

In [10]:
sum(labels)

19209762

In [11]:
unique_labels = np.unique(labels)

In [12]:
len(unique_labels)

1504

### Read entire rows

A single row can be accessed using the [`.loc[]` operator](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.loc.html):

In [13]:
row2 = md10.loc[2]; print(row2)

mapobject_id        491299
plate_name              p1
well_name              D10
well_pos_y               0
well_pos_x               0
tpoint                   0
zplane                   0
label                    3
is_border                1
Classification-5         0
TPlus                    0
Name: 2, dtype: object


Each row acts as special array, where you can access items by column name *or* by position:

In [14]:
row2['well_name']

'D10'

In [15]:
row2[2]

'D10'

## Quick computation of basic statistical quantities

A `DataFrame`'s `.describe()` method provides a quick statistical summary of the data (but only for *continuous* variables):

In [16]:
stat = fv10['Intensity_mean_A01_C03'].describe()

print(stat)

count    30764.000000
mean       183.510022
std         16.797994
min        107.724627
25%        172.740721
50%        182.523436
75%        193.056764
max        367.000000
Name: Intensity_mean_A01_C03, dtype: float64


Note that the value returned by `.describe()` is again a kind of "row" object, and you can access individual fields with the `[]` operator:

In [17]:
stat['mean'] + stat['std']

200.3080152385092

You can compute the very same values (and many others!) by directly applying a NumPy function to the column:

In [18]:
# compute the median
np.median(fv10['Intensity_mean_A01_C03'])

182.523436

In [19]:
# compute the average of the log intensity
np.mean(np.log(fv10['Intensity_mean_A01_C03']))

5.20817002879652

Note that if a categorical variable (e.g., a yes/no flag) is encoded using numerical values (e.g., `0` and `1`), then PanDas will still happily provide a statistical summary, like it were a continuous variable, but it will *not* be much useful:

In [20]:
md10['TPlus'].describe()

count    30764.000000
mean         0.150338
std          0.357408
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: TPlus, dtype: float64

-----

## Exercise 8.A

Load "metadata" files into tables `md07` and `md10`.

Then load "feature values" files into tables `fv07` and `fv10`.

How many rows are in each table?

In [21]:
# your solution here

## Exercise 8.B

What is the mean value of column `Intensity_mean_A01_C03` in each well?

In [22]:
# your solution here

## Exercise 8.C

How many unique values are in column `TPlus` in the metadata of each well?
*(Requires some familiary with NumPy)*

In [23]:
# your solution here

## Exercise 8.D

The `is_border` column in a "metadata" table tells you whether a cell lies at the border of an acquisition site or not (1 = lies at the border, 0 = does not touch nor cross the border).

Can you count the number of "border" cells in each well?

In [24]:
# your solution here

-----

## Join tabular data

There are two ways of joining tables:

- we may want to "stack" one table on top of another: this is only possible if the tables have the same columns, and is accomplished with PanDas' function [concat()](https://pandas.pydata.org/pandas-docs/stable/merging.html#concatenating-objects), or

- we may want to form a new table by "joining" rows that have some value (called a *key*) in common; this is accomplished by a `DataFrame` [`.merge()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)

### "Stacking" tables

Stacking tables is accomplished with PanDas' function [concat()](https://pandas.pydata.org/pandas-docs/stable/merging.html#concatenating-objects), which takes a unique argument: a *list* of tables to stack one on top of the other.

In [25]:
md = pd.concat([md07, md10])

NameError: name 'md07' is not defined

In [ ]:
fv = pd.concat([fv07, fv08, fv09, fv10])

### Joining by rows

The [`.merge()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) allows making a new table out of two existing tables with a set of common columns.  The syntax is as follows:
```
new_table = table1.merge(table2, how, on=[list of column names])
```

For example:

In [ ]:
merged10 = md10.merge(fv10, how='inner', on=['mapobject_id'])

In [ ]:
merged10.head()

Parameter *how* has the following possible values:

* `'inner'`: merge rows where both tables agree on the value of the common columns;
* `'left'`: take _all_ rows from `table1`, fill in null values if there's no row in `table2` which agrees on the common columns;
* `'right'`: same as `'left'` but with the roles of `table1` and `table2` interchanged;
* `'outer'`: take _all_ rows, fill in null values when row of one table cannot be matched.

If you know databases, this is exactly the same semantics of [SQL's JOIN keyword](https://www.w3schools.com/sql/sql_join.asp)

## Make sub-tables

The [`.loc[]` operator](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.loc.html) can be used to extract sub-tables, by specifying the rows and columns to extract:
```
sub = data.loc[rows, columns]
```
here `rows` and `columns` can be:

- *lists* of row numbers (e.g., `[1,2,3]`) or column names (e.g., `['a', 'c']`), 
- *ranges* (e.g., `1:10`),
- or **selectors** (which we'll explain in a while)

For example:

In [ ]:
md10.loc[[1,3,5,7], ['well_name', 'label', 'is_border']]

**Note:** the `.loc[]` operator does *not copy* the table data -- so any modification to the sub-table will be reflected on the parent table.  Use `.loc[...].copy()` to create an independent sub-table.

## Selectors

Selectors are logical expressions on a column (or row) of a table.

In [ ]:
# select all objects in well D10 with numerical label 42
label42 = (md10['label'] == 42)

In [ ]:
# select all objects whose mean intensity in the DAPI channel is over 1 sigma from the avg 
dapi_plus1s = (fv10['Nuclei_Intensity_mean_A02_C01'] > 259.69)

You can use selectors to extract a sub-table out of an existing one.

In [ ]:
# only data about cell labeled 42
table_only_label42 = md10.loc[label42]

In [ ]:
len(table_only_label42)

In [ ]:
table_only_label42.head()

-----

## Exercise 5. 

Make stacked tables:

- `md`, combining metadata for all wells
- `fv`, combining feature values for all wells

In [ ]:
# your solution here

## Exercise 6.

Make a single large table `all` by joining tables `md` and `fv` over the common column `mapobject_id`.

How many rows are in the combined table?

In [ ]:
# your solution here

## Exercise 7.

Make a table `good` by extracting from `all` only rows which refer to objects that are *not* "border" objects.

How many good objects are there?  

*Bonus points:* could you compute this number from the selector alone?

In [ ]:
# your solution

## Exercise 8.

Make two tables `md0` and `md1` by splitting on the two values of column `TPlus` (`0` or `1`).
What is the mean of column `Intensity_mean_A01_C03` in each table?  And the std deviation?

In [ ]:
# your solution

-----

# Plotting

The [seaborn](https://seaborn.pydata.org/examples/index.html) library provides many convenient plotting functions; there are good chances that the plot you want to make is already implemented by Seaborn.

## Distribution plots (with kernel density estimates)

Seaborn's function [distplot()](https://seaborn.pydata.org/generated/seaborn.distplot.html#seaborn.distplot) plots the frequency distribution of an array of values.

The simplest use just passes an array of values (e.g., a `DataFrame` column) as the unique argument:

In [ ]:
sns.distplot(fv10['Intensity_mean_A01_C03'])

The plot ink color can be specified with the additional parameter `color=`:

In [ ]:
sns.distplot(fv10['Nuclei_Intensity_mean_A02_C01'], color='orange')

-----

## Exercise 9.

Make a distribution plots of the mean Intensity of the *Yap* channel for each of the wells D07, D08, D09, D10. Plot each well in a different color.

-----

## Box plots

Box plots are provided by [Seaborn function violinplot()](https://seaborn.pydata.org/generated/seaborn.boxplot.html#seaborn.violinplot).  The syntax of the `violinplot()` function we are going to use is the following:
```
boxplot(data=..., x=..., y=..., hue=...)
```
where the named arguments have the following meaning:
- `data` is a table (PanDas `DataFrame`)
- `x` is the *name of a column* of `data` where to draw the x-axis values from; this should be a categorical variable.
- `y` is again the *name of a column* whose distribution detemines the parameters of the box and handles; `y` should name a continuous variable.

The following are optional:
- `hue` is again the *name of a column* of `data` (carrying a categorical variable): if supplied, for each value of `x`, a box+handles of a different color ("hue") will be drawn for each value of `hue`. In other words, by supplying `hue` you can make boxplots of 3D-data `(x,y,hue)`.
- `ax` used for placing a plot on a specific "subplot"

In [ ]:
# Initialize the matplotlib figure
fig, ax = plt.subplots(1, figsize=(4, 3))

sns.boxplot(data=merged2, x='well_name', hue='TPlus', y='Intensity_mean_A01_C03', ax=ax)

-----

## Exercise 10.

Make a 2x2 grid of plots, showing box plots of the mean Intensity of the *Yap* channel for each of the wells D07, D08, D09, D10.

-----

## Violin plots

Violin plots are provided by [Seaborn function violinplot()](https://seaborn.pydata.org/generated/seaborn.violinplot.html#seaborn.violinplot).  Producing a (useful) violin plot requires specifying quite some parameters; the syntax of the `violinplot()` function we are going to use is the following:
```
violinplot(data=..., x=..., y=..., hue=..., split=..., inner=...)
```
where the named arguments have the following meaning:
- `data` is a table (PanDas `DataFrame`)
- `x` is the *name of a column* of `data` where to draw the x-axis values from; this should be a categorical variable.
- `y` is again the *name of a column* whose distribution will be plotted to make the countour of the "violins"; `y` should name a continuous variable.

The following are optional:
- `hue` is again the *name of a column* of `data` (carrying a categorical variable): if supplied, for each value of `x`, a violin of a different color ("hue") will be drawn for each value of `hue`. In other words, by supplying `hue` you can make violin plots of 3D-data `(x,y,hue)`.
- `split`: if `True` and `hue` is the name of a discrete variable with 2 values, then, for each `x`, each half of the violin plots the `y` values corresponding to the different `hue`.
- `inner`: can be one of:
  * `'quartiles'`: mark the quartiles of the distribution
  * `'box'` make a miniature box plot inside the violin
  * `'point'` mark each data point
  * `None`: no inner decoration of the "violins"

In [ ]:
sns.violinplot(data=merged2, x='well_name', y='Intensity_mean_A01_C03', hue='TPlus')

-----

## Exercise 11.

Make a violin plot comparing the distribution of *mean Intensity* for channel *A01_C03* side-by-side for cells with TPlus=0 and TPlus=1.

## Exercise 12.

Make a 2x2 grid of the above violin plots, allowing to compare across wells.

-----